In [1]:
import tensorflow as tf
import cv2
from pixel2mesh.models import GCN
from pixel2mesh.fetcher import *
from tqdm import tqdm
import time
import pandas as pd

# Set random seed
seed = 1024
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('data_list', 'pixel2mesh/hitoshi_utils/train_list.txt', 'Data list path.')
flags.DEFINE_float('learning_rate', 3e-5, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 30, 'Number of epochs to train.')
flags.DEFINE_integer('hidden', 192, 'Number of units in  hidden layer.')
flags.DEFINE_integer('feat_dim', 963, 'Number of units in perceptual feature layer.')
flags.DEFINE_integer('coord_dim', 3, 'Number of units in output layer.') 
flags.DEFINE_float('weight_decay', 5e-6, 'Weight decay for L2 loss.')

In [2]:
# Define placeholders(dict) and model
num_blocks = 3
num_supports = 2
placeholders = {
    'features': tf.placeholder(tf.float32, shape=(None, 3)), # initial 3D coordinates
    'img_inp': tf.placeholder(tf.float32, shape=(224, 224, 3)), # input image to network
    'labels': tf.placeholder(tf.float32, shape=(None, 6)), # ground truth (point cloud with vertex normal)
    'support1': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the first block
    'support2': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the second block
    'support3': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the third block
    'faces': [tf.placeholder(tf.int32, shape=(None, 4)) for _ in range(num_blocks)], # helper for face loss (not used)
    'edges': [tf.placeholder(tf.int32, shape=(None, 2)) for _ in range(num_blocks)], # helper for normal loss
    'lape_idx': [tf.placeholder(tf.int32, shape=(None, 10)) for _ in range(num_blocks)], # helper for laplacian regularization
    'pool_idx': [tf.placeholder(tf.int32, shape=(None, 2)) for _ in range(num_blocks-1)] # helper for graph unpooling
}
model = GCN(placeholders, logging=True)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [3]:
def construct_feed_dict(pkl, placeholders):
    coord, pool_idx, faces, lape_idx,  = pkl[0], pkl[4], pkl[5], pkl[7]
    edges = []
    for i in range(1,4):
        adj = pkl[i][1]
        edges.append(adj[0])
    feed_dict = dict()
    feed_dict.update({placeholders['features']: coord})
    feed_dict.update({placeholders['edges'][i]: edges[i] for i in range(len(edges))})
    feed_dict.update({placeholders['faces'][i]: faces[i] for i in range(len(faces))})
    feed_dict.update({placeholders['pool_idx'][i]: pool_idx[i] for i in range(len(pool_idx))})
    feed_dict.update({placeholders['lape_idx'][i]: lape_idx[i] for i in range(len(lape_idx))})
    feed_dict.update({placeholders['support1'][i]: pkl[1][i] for i in range(len(pkl[1]))})
    feed_dict.update({placeholders['support2'][i]: pkl[2][i] for i in range(len(pkl[2]))})
    feed_dict.update({placeholders['support3'][i]: pkl[3][i] for i in range(len(pkl[3]))})
    return feed_dict

In [4]:
# Load data
data = DataFetcher(FLAGS.data_list)
data.setDaemon(True)
data.start()
train_number = data.number
# Initialize session
config=tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
model.load(sess)
# Construct feed dictionary
pkl = pickle.load(open('pixel2mesh/utils/ellipsoid/info_ellipsoid.dat', 'rb'))
feed_dict = construct_feed_dict(pkl, placeholders)

# Train model
train_loss = open('pixel2mesh/utils/record_training_loss.log', 'a')
train_loss.write('Start training, lr =  %f\n'%(FLAGS.learning_rate))

INFO:tensorflow:Restoring parameters from pixel2mesh/hitoshi_utils/hitoshi_checkpoint/gcn.ckpt
Model restored from file: pixel2mesh/hitoshi_utils/hitoshi_checkpoint/gcn.ckpt


In [ ]:
start = time.time()
for epoch in tqdm(range(FLAGS.epochs)):
    all_loss = np.zeros(train_number, dtype='float32') 
    for iters in range(train_number):
        img_inp, y_train, data_id = data.fetch()
        feed_dict.update({placeholders['img_inp']: img_inp})
        feed_dict.update({placeholders['labels']: y_train})
        _, dists,out1,out2,out3 = sess.run([model.opt_op,model.loss,model.output1,model.output2,model.output3], feed_dict=feed_dict)
        all_loss[iters] = dists
        mean_loss = np.mean(all_loss[np.where(all_loss)])
        if (iters+1) % 32 == 0:
            print 'Epoch %d, Iteration %d'%(epoch + 1,iters + 1)
            print 'Mean loss = %f, iter loss = %f, %d'%(mean_loss,dists,data.queue.qsize())
    model.save(sess)
    train_loss.write('Epoch %d, loss %f\n'%(epoch+1, mean_loss))
    train_loss.flush()

end = time.time()
print(end - start)
pd.DataFrame([{'runtime':end-start}]).to_csv('runtime.csv')
#data.shutdown()
#print 'Training Finished!'


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1, Iteration 32
Mean loss = 3.974855, iter loss = 3.330473, 64
Epoch 1, Iteration 64
Mean loss = 4.323114, iter loss = 4.754881, 64
Epoch 1, Iteration 96
Mean loss = 4.258914, iter loss = 4.370028, 64
Epoch 1, Iteration 128
Mean loss = 4.280997, iter loss = 5.690005, 64
Epoch 1, Iteration 160
Mean loss = 4.344029, iter loss = 2.809254, 64
Epoch 1, Iteration 192
Mean loss = 4.401898, iter loss = 4.366717, 64
Epoch 1, Iteration 224
Mean loss = 4.501796, iter loss = 3.096311, 64
Epoch 1, Iteration 256
Mean loss = 4.483405, iter loss = 4.553914, 64
Epoch 1, Iteration 288
Mean loss = 4.523643, iter loss = 4.026444, 64
Epoch 1, Iteration 320
Mean loss = 4.544886, iter loss = 4.072260, 64
Epoch 1, Iteration 352
Mean loss = 4.513809, iter loss = 4.053320, 64
Epoch 1, Iteration 384
Mean loss = 4.480497, iter loss = 6.032156, 64
Epoch 1, Iteration 416
Mean loss = 4.447428, iter loss = 6.507195, 64
Epoch 1, Iteration 448
Mean loss = 4.436761, iter loss = 3.804459, 64
Epoch 1, Iteration 480


Epoch 1, Iteration 3744
Mean loss = 4.357165, iter loss = 4.357910, 64
Epoch 1, Iteration 3776
Mean loss = 4.356093, iter loss = 1.812011, 64
Epoch 1, Iteration 3808
Mean loss = 4.356217, iter loss = 5.514249, 64
Epoch 1, Iteration 3840
Mean loss = 4.355319, iter loss = 3.653419, 64
Epoch 1, Iteration 3872
Mean loss = 4.349965, iter loss = 4.530790, 64
Epoch 1, Iteration 3904
Mean loss = 4.351745, iter loss = 4.465195, 64
Epoch 1, Iteration 3936
Mean loss = 4.348769, iter loss = 4.918640, 64
Epoch 1, Iteration 3968
Mean loss = 4.346288, iter loss = 5.660114, 64
Epoch 1, Iteration 4000
Mean loss = 4.344004, iter loss = 4.384277, 64
Epoch 1, Iteration 4032
Mean loss = 4.344598, iter loss = 5.952067, 64
Epoch 1, Iteration 4064
Mean loss = 4.347536, iter loss = 2.000691, 64
Epoch 1, Iteration 4096
Mean loss = 4.345976, iter loss = 6.364096, 64
Epoch 1, Iteration 4128
Mean loss = 4.345581, iter loss = 4.253821, 64
Epoch 1, Iteration 4160
Mean loss = 4.347389, iter loss = 4.179351, 64
Epoch 

Epoch 1, Iteration 7456
Mean loss = 4.337631, iter loss = 4.062385, 64
Epoch 1, Iteration 7488
Mean loss = 4.337389, iter loss = 4.780004, 64
Epoch 1, Iteration 7520
Mean loss = 4.336463, iter loss = 3.385868, 64
Epoch 1, Iteration 7552
Mean loss = 4.337518, iter loss = 4.936088, 64
Epoch 1, Iteration 7584
Mean loss = 4.337871, iter loss = 3.332464, 64
Epoch 1, Iteration 7616
Mean loss = 4.336175, iter loss = 3.707994, 64
Epoch 1, Iteration 7648
Mean loss = 4.336865, iter loss = 4.676832, 64
Epoch 1, Iteration 7680
Mean loss = 4.337692, iter loss = 3.570105, 64
Epoch 1, Iteration 7712
Mean loss = 4.337537, iter loss = 5.045092, 64
Epoch 1, Iteration 7744
Mean loss = 4.337978, iter loss = 2.799595, 64
Epoch 1, Iteration 7776
Mean loss = 4.338326, iter loss = 2.913342, 64
Epoch 1, Iteration 7808
Mean loss = 4.338195, iter loss = 5.223792, 64
Epoch 1, Iteration 7840
Mean loss = 4.338804, iter loss = 3.187305, 64
Epoch 1, Iteration 7872
Mean loss = 4.338085, iter loss = 3.884281, 64
Epoch 

Epoch 1, Iteration 11136
Mean loss = 4.332690, iter loss = 3.624472, 64
Epoch 1, Iteration 11168
Mean loss = 4.331234, iter loss = 2.228209, 64
Epoch 1, Iteration 11200
Mean loss = 4.330791, iter loss = 4.162835, 64
Epoch 1, Iteration 11232
Mean loss = 4.330626, iter loss = 5.111677, 64
Epoch 1, Iteration 11264
Mean loss = 4.331301, iter loss = 4.445998, 64
Epoch 1, Iteration 11296
Mean loss = 4.331774, iter loss = 4.006253, 64
Epoch 1, Iteration 11328
Mean loss = 4.331688, iter loss = 4.904082, 64
Epoch 1, Iteration 11360
Mean loss = 4.331932, iter loss = 4.015073, 64
Epoch 1, Iteration 11392
Mean loss = 4.331778, iter loss = 3.208851, 64
Epoch 1, Iteration 11424
Mean loss = 4.331020, iter loss = 6.501317, 64
Epoch 1, Iteration 11456
Mean loss = 4.331688, iter loss = 5.287392, 64
Epoch 1, Iteration 11488
Mean loss = 4.331419, iter loss = 4.025428, 64
Epoch 1, Iteration 11520
Mean loss = 4.330969, iter loss = 4.113001, 64
Epoch 1, Iteration 11552
Mean loss = 4.330899, iter loss = 4.262

...

Epoch 1, Iteration 69504
Mean loss = 4.390928, iter loss = 5.307023, 64
Epoch 1, Iteration 69536
Mean loss = 4.390966, iter loss = 4.013270, 64
Epoch 1, Iteration 69568
Mean loss = 4.391054, iter loss = 2.333638, 64
Epoch 1, Iteration 69600
Mean loss = 4.391120, iter loss = 3.995259, 64
Epoch 1, Iteration 69632
Mean loss = 4.391123, iter loss = 5.115981, 64
Epoch 1, Iteration 69664
Mean loss = 4.391172, iter loss = 5.314818, 64
Epoch 1, Iteration 69696
Mean loss = 4.390948, iter loss = 4.481718, 64
Epoch 1, Iteration 69728
Mean loss = 4.391098, iter loss = 4.145974, 64
Epoch 1, Iteration 69760
Mean loss = 4.391133, iter loss = 3.497212, 64
Epoch 1, Iteration 69792
Mean loss = 4.390995, iter loss = 6.072798, 64
Epoch 1, Iteration 69824
Mean loss = 4.390961, iter loss = 3.934384, 64
Epoch 1, Iteration 69856
Mean loss = 4.391099, iter loss = 2.679746, 64
Epoch 1, Iteration 69888
Mean loss = 4.391136, iter loss = 3.609880, 64
Epoch 1, Iteration 69920
Mean loss = 4.391192, iter loss = 4.500

Epoch 1, Iteration 73152
Mean loss = 4.391279, iter loss = 4.394626, 64
Epoch 1, Iteration 73184
Mean loss = 4.391429, iter loss = 3.879777, 64
Epoch 1, Iteration 73216
Mean loss = 4.391384, iter loss = 3.702710, 64
Epoch 1, Iteration 73248
Mean loss = 4.391492, iter loss = 4.465799, 64
Epoch 1, Iteration 73280
Mean loss = 4.391590, iter loss = 5.309203, 64
Epoch 1, Iteration 73312
Mean loss = 4.391882, iter loss = 6.657510, 64
Epoch 1, Iteration 73344
Mean loss = 4.391911, iter loss = 2.975657, 64
Epoch 1, Iteration 73376
Mean loss = 4.391962, iter loss = 3.815511, 64
Epoch 1, Iteration 73408
Mean loss = 4.392032, iter loss = 4.216464, 64
Epoch 1, Iteration 73440
Mean loss = 4.392086, iter loss = 5.205852, 64
Epoch 1, Iteration 73472
Mean loss = 4.392036, iter loss = 2.107490, 64
Epoch 1, Iteration 73504
Mean loss = 4.392088, iter loss = 5.443240, 64
Epoch 1, Iteration 73536
Mean loss = 4.392214, iter loss = 4.151855, 64
Epoch 1, Iteration 73568
Mean loss = 4.392282, iter loss = 3.982

Epoch 1, Iteration 76800
Mean loss = 4.394295, iter loss = 5.551410, 64
Epoch 1, Iteration 76832
Mean loss = 4.394279, iter loss = 2.123190, 64
Epoch 1, Iteration 76864
Mean loss = 4.394364, iter loss = 6.257693, 64
Epoch 1, Iteration 76896
Mean loss = 4.394407, iter loss = 3.789317, 64
Epoch 1, Iteration 76928
Mean loss = 4.394351, iter loss = 2.155806, 64
Epoch 1, Iteration 76960
Mean loss = 4.394355, iter loss = 6.074442, 64
Epoch 1, Iteration 76992
Mean loss = 4.394496, iter loss = 3.815516, 64
Epoch 1, Iteration 77024
Mean loss = 4.394578, iter loss = 5.331991, 64
Epoch 1, Iteration 77056
Mean loss = 4.394524, iter loss = 3.375401, 64
Epoch 1, Iteration 77088
Mean loss = 4.394623, iter loss = 5.141216, 64
Epoch 1, Iteration 77120
Mean loss = 4.394777, iter loss = 5.402917, 64
Epoch 1, Iteration 77152
Mean loss = 4.394847, iter loss = 5.823783, 64
Epoch 1, Iteration 77184
Mean loss = 4.394980, iter loss = 3.939220, 64
Epoch 1, Iteration 77216
Mean loss = 4.395086, iter loss = 3.570

Epoch 1, Iteration 80448
Mean loss = 4.394771, iter loss = 2.316827, 64
Epoch 1, Iteration 80480
Mean loss = 4.394878, iter loss = 3.352981, 64
Epoch 1, Iteration 80512
Mean loss = 4.394970, iter loss = 2.768907, 64
Epoch 1, Iteration 80544
Mean loss = 4.395015, iter loss = 5.550391, 64
Epoch 1, Iteration 80576
Mean loss = 4.395067, iter loss = 4.405548, 64
Epoch 1, Iteration 80608
Mean loss = 4.394974, iter loss = 3.138355, 64
Epoch 1, Iteration 80640
Mean loss = 4.394901, iter loss = 2.693252, 64
Epoch 1, Iteration 80672
Mean loss = 4.394832, iter loss = 5.063328, 64
Epoch 1, Iteration 80704
Mean loss = 4.394750, iter loss = 4.025387, 64
Epoch 1, Iteration 80736
Mean loss = 4.394715, iter loss = 4.268461, 64
Epoch 1, Iteration 80768
Mean loss = 4.394743, iter loss = 3.728486, 64
Epoch 1, Iteration 80800
Mean loss = 4.394667, iter loss = 6.695841, 64
Epoch 1, Iteration 80832
Mean loss = 4.394652, iter loss = 2.111667, 64
Epoch 1, Iteration 80864
Mean loss = 4.395023, iter loss = 5.106

Epoch 1, Iteration 84096
Mean loss = 4.398171, iter loss = 3.549546, 64
Epoch 1, Iteration 84128
Mean loss = 4.398263, iter loss = 4.501942, 64
Epoch 1, Iteration 84160
Mean loss = 4.398284, iter loss = 1.656553, 64
Epoch 1, Iteration 84192
Mean loss = 4.398313, iter loss = 3.851720, 64
Epoch 1, Iteration 84224
Mean loss = 4.398265, iter loss = 3.154747, 64
Epoch 1, Iteration 84256
Mean loss = 4.398256, iter loss = 4.245947, 64
Epoch 1, Iteration 84288
Mean loss = 4.398299, iter loss = 4.255098, 64
Epoch 1, Iteration 84320
Mean loss = 4.398285, iter loss = 4.758645, 64
Epoch 1, Iteration 84352
Mean loss = 4.398387, iter loss = 5.733873, 64
Epoch 1, Iteration 84384
Mean loss = 4.398546, iter loss = 4.206449, 64
Epoch 1, Iteration 84416
Mean loss = 4.398489, iter loss = 4.637777, 64
Epoch 1, Iteration 84448
Mean loss = 4.398593, iter loss = 3.909199, 64
Epoch 1, Iteration 84480
Mean loss = 4.398702, iter loss = 3.202918, 64
Epoch 1, Iteration 84512
Mean loss = 4.398859, iter loss = 8.846